# Házi feladat



Írjunk egy `Permutation` osztályt. Permutáció alatt most egy
$\left\{{0,\dots,n-1}\right\}\to\left\{{0,\dots,n-1}\right\}$ bijekciót
értünk! Inicializáláskor a $\pi$ permutációt az
$(\pi(0), \dots,\pi(n-1))$ rendezett $n$-sel adjuk meg.

Írjuk meg az `__str__` és `__repr__` metódusokat. A `__str__` metódus a
permutáció ciklus reprezentációját adja vissza a `cycles: ` szöveg után.

A $\pi$ permutáció egy ciklusa alatt egy $c_0,\dots, c_{k-1}$ sorozatot
értünk, ahol $\pi(c_t)=c_{t+1}$, ha $t+1<k$ és $\pi(c_{k-1})=c_0$. Ha a
permutációt írányított gráfként ábrázoljuk ($i\to \pi(i)$ élekkel),
akkor ezek a keletkező körök. A ciklusok az alaphalmazt ekvivalencia
osztályokra bontják, a permutáció megadásához elgendő a ciklusokon belül
feljegyezni a sorrendet. Sőt az egy hosszú ciklusokra nincs is szükség a
leíráshoz. [További információért, lásd a Wikipédia
bejegyzést!](https://en.wikipedia.org/wiki/Cycles_and_fixed_points)

Minden egyes ciklust a legkisebb elemével kezdve írjunk fel, és a
ciklusok sorrendjét is a legkisebb elemek sorrendje határozza meg. Nem
kell túlbonyolítani a dolgot. Ha a ciklusokat már kiszámoltuk, akkor a
ciklusokból álló lista rendezése pont ezt csinálja! Példák.

```python
(0, 1, 2, 3, 4, 5): ciklus reprezentáció = []
(0, 1, 2, 3, 5, 4): ciklus reprezentáció = [(4,5)]
(1, 2, 3, 4, 5, 0): ciklus reprezentáció = [(0, 1, 2, 3, 4, 5)]
(1, 2, 0, 4, 5, 3): ciklus reprezentáció = [(0, 1, 2), (3, 4, 5)]
```

A `__repr__` metódus a szokásos alakú, osztály név és az inicializáló
argumentumok.

Példák.

```python
pi0 = Permutation([1, 2, 3, 0])
print(pi0)       # -> cycles : [(0, 1, 2, 3)]
print(repr(pi0)) # -> Permutation((1, 2, 3, 0))
pi1 = Permutation([1, 0, 3, 2])
print(pi1)       # -> cycles : [(0, 1), (2, 3)]
print(repr(pi1)) # -> Permutation((1, 0, 3, 2))
```

Feltehetjük, hogy az osztályt mindig helyesen használják, azaz új egyed
létrehozásakor a bemenet mindig egy permutáció.

Implementáljuk a szorzás műveletet permutációk között. A szorzás a
kompozió műveletet jelenti. Azaz $\pi_1\pi_2(i) = \pi_1(\pi_2(i))$.

A `~` operátor az inverz permutációt adja vissza. Az inverz permutáció
az a permutáció, amelyre $\pi\pi^{-1} = \pi^{-1}\pi = \text{id}$, ahol
$\text{id}$ az identitás permutáció. A `~` művelet a `__invert__` dunder
metódust használja.

A `Permutation` osztály egyedein működjön a `len` függvény és az
indexelés. Azaz ha $\pi$ permutáció, akkor $\pi[i]$ az $i$-edik elemét
adja vissza a permutációnak.

Egy permutáció legyen függvény is. Ilyenkor a bemenetként kapott $n$
hosszú sorozatot permutálja. A kimenet $i$-edik eleme a bemenet
$\pi(i)$-edik eleme. Meg tudjuk-e oldani, hogy a visszaadott érték olyan
típusú legyen, mint a bemenet?
```python
p = Permutation([1, 0, 2])
print(p("abc")) # -> bca
print(p([11, 2, 3])) # -> [2, 3, 11]
```

Feltehető, hogy a bemenet vagy 'list', vagy 'tuple', vagy 'str', vagy
'Permutation' típusú.

Szokás szerint írjunk teszt függvényt, ami az osztály metódusait
ellenőrzi 4-5 teszt eseten. A teszteléshez használjuk az `ipytest`
könytárat.

In [ ]:
import graphviz

In [ ]:
class Permutation:
    
    def __init__(self, perm):
        try:
            self.permutation = tuple(perm)
        except TypeError:
            raise TypeError("not a sequence")
        if sorted(self.permutation) != list(range(len(self.permutation))):
            raise ValueError("not a permutation")
        self._cycles = None

    def __getitem__(self, key):
        return self.permutation[key]
    
    def __len__(self):
        return len(self.permutation)
    
    def __mul__(self, other):
        if not isinstance(other, type(self)):
            raise TypeError("can only multiply permutations")
        if len(self) != len(other):
            raise ValueError("permutations have different lengths")
        return Permutation(self.permutation[a] for a in other)

    def __truediv__(self, other):
        return self * (~other)

    def __invert__(self):
        inv = [0]*len(self.permutation)
        for a, b in enumerate(self.permutation):
            inv[b] = a
        return Permutation(inv)

    def __pow__(self, n):
        result = Permutation(range(len(self.permutation)))
        for _ in range(abs(n)):
            result *= self
        return result if n >= 0 else ~result

    def _get_cycles(self):
        cycles = []
        pi = list(self.permutation)
        for i, x in enumerate(pi):
            if (x == -1) or (i == x):
                continue
            cycle = [i]
            while x != i:
                cycle.append(x)
                pi[x], x = -1, pi[x]
            cycles.append(tuple(cycle))
        return cycles

    def __str__(self):
        if self._cycles is None:
            self._cycles = self._get_cycles()
        return f"cycles: {self._cycles}"

    def __repr__(self):
        return f"{type(self).__name__}({self.permutation})"

    def __eq__(self, other):
        return isinstance(other, type(self)) and other.permutation == self.permutation

In [ ]:
import ipytest
ipytest.autoconfig()

In [ ]:
%%ipytest
from pytest import raises
import itertools

def test_init():
    with raises(TypeError):
        Permutation(1)

    with raises(ValueError):
        Permutation("abc")

    with raises(ValueError):
        Permutation((1, 2))

    with raises(ValueError):
        Permutation((0, 0.2))

    assert Permutation((0,1,2,3)).permutation == (0,1,2,3)

    seq = [0, 1, 2]
    p = Permutation(seq)
    seq[0] = 4
    assert p.permutation == (0, 1, 2)

def test_mul():
    p = Permutation((1, 2, 0, 3, 4))
    assert p*p == Permutation((2, 0, 1, 3, 4))


def test_div():
    p = Permutation((1, 2, 0, 3, 4))
    q = Permutation((0, 1, 2, 4, 3))
    assert p/q == Permutation((1, 2, 0, 4, 3))

def test_inv():
    identity = Permutation(range(5))
    for p in itertools.permutations(range(5)):
        pi = Permutation(p)
        assert pi*(~pi) == identity

def test_str():
    identity = Permutation(range(5))
    assert str(identity) == "cycles: []"

    p = Permutation((1, 2, 0, 3, 4))
    assert str(p) == "cycles: [(0, 1, 2)]"


def test_repr():
    identity = Permutation(range(5))
    assert repr(identity) == "Permutation((0, 1, 2, 3, 4))"

    p = Permutation((1, 2, 0, 3, 4))
    assert repr(p) == "Permutation((1, 2, 0, 3, 4))"



### Vizualizáció

Gráfok gyakran előfordulnak és jó lenne őket ábrázolni. Erre szolgál a [`graphviz` python](https://graphviz.readthedocs.io/) könyvtár.
Ez  python könyvtár az ugyanilyen nevű `C` könyvtárat használja. Azt is le kell tölteni, ha nincs telepítve a 
[graphviz](https://www.graphviz.org/)

In [ ]:
try:
    import graphviz
except ModuleNotFoundError:
    print("On ubuntu/debian try:")
    print("!pip install graphviz")
    print("!sudo apt install graphviz")
    print('Then try again!')

Ezután egy nem túl nagy irányított gráfot, pl. egy véleltlen permutációból kapottat a következőképpen jeleníthetünk meg

In [ ]:
import random

permutation = list(range(8))
print(f"permutation before shuffling: {permutation}")
random.shuffle(permutation)
print(f" permutation after shuffling: {permutation}")

## making edges for graphviz. Node labels must be strings
edges = [(str(head), str(tail)) for head, tail in enumerate(permutation)]
G = graphviz.Digraph()
G.edges(edges)

display(G)

pi = Permutation(permutation)
print({str(pi)}, {repr(pi)})

In [ ]:
print(G.pipe(format='dot').decode('utf8'))

## Vizualizáció hozzáadása a Permutation osztályhoz.

In [ ]:
def repr_svg(self):
    graph = graphviz.Digraph()
    graph.edges((str(a), str(b)) for a, b in enumerate(self))
    return graph._repr_image_svg_xml()

Permutation._repr_svg_ = repr_svg


In [ ]:
import gravis

In [ ]:
def repr_html(self):
    data = {
        "graph": {
            "directed": True,
            "nodes": [{"label": str(i)} for i in range(len(self))],
            "edges": [{"source": str(a), "target": str(b)} for a, b in enumerate(self)]
        }
    }
    return gravis.d3(data).to_html_partial()

Permutation._repr_html_ = repr_html
Permutation((1, 2, 0, 3, 4))

## Snapshot létrehozása

In [ ]:
from pathlib import Path
import json

path = Path("data")
if not path.exists():
    path.mkdir()
    
snapshot_file = path / "snapshot.json"


if not snapshot_file.exists():
    snapshot = [
        {
            "permutation": p,
            "svg_string": Permutation(p)._repr_svg_()
        }
        for p in [(0, 1, 2, 3, 4),  (1, 2, 0)]
    ]
    with open(snapshot_file, "w") as file:
        json.dump(snapshot, file, indent=2)

    print(f"snapshot is written into {snapshot_file.absolute()}")


In [ ]:
!cat data/snapshot.json

In [ ]:
delattr(Permutation, "_repr_html_")
pi = Permutation((1,2,3,0,4))
pi

In [ ]:
%%ipytest 
import json

def test_svg():
    with open("data/snapshot.json", "r") as file:
        examples = json.load(file)
    for example in examples:
        seq = example['permutation']
        svg = example['svg_string']
        pi = Permutation(seq)
        assert pi._repr_svg_() == svg


## További tesztelési lehetőség

Készítsünk véletlen ciklus felbontást, és számítsuk ki a hozzá tartozó permutációt! A két függvényben 
`permutáció -> ciklus felbontás` és `ciklus felbontás -> permutáció` valószínűleg nem követjük el ugyanazt a hibát! 

In [ ]:
import random
import itertools

def cycle_to_perm(cycles, n):
    perm = list(range(n))
    for cycle in cycles:
        for x, y in itertools.pairwise(cycle):
            perm[x] = y 
        perm[cycle[-1]] = cycle[0]
    return perm

def random_cycles(n):
    ## not efficient in theory!
    points = list(range(n))
    cycles = []
    while points:
        i = points.pop(0)
        k = random.randint(0, len(points))
        if k>0:
            c = [i]
            for _ in range(k):
                j = random.randrange(len(points))
                c.append(points.pop(j))
            cycles.append(tuple(c))

    return cycles
        


In [ ]:
def pop(lst, i, n):
    if i == n-1:
        return lst[i]
        
    value = lst[i]
    lst[i] = lst[n-1]
    root = i
    while root < n:
        new_root = root 
        child = 2*root+1
        
        if child < n and lst[child] < lst[new_root]:
            new_root = child
        child = child+1
        
        if child < n and lst[child] < lst[new_root]:
            new_root = child

        if root == new_root:
            break
        
        lst[root], lst[new_root] = lst[new_root], lst[root]
        root = new_root
        
    return value

def random_cycles2(n):
    ## more efficient in theory
    points = list(range(n))
    cycles = []
    while n>0:
        k = random.randrange(0, n)
        c = [pop(points, 0, n)]
        n -= 1
        for _ in range(k):
            j = random.randrange(n)
            c.append(pop(points, j ,n))
            n -= 1
        if len(c)>1:
            cycles.append(tuple(c))

    return cycles


In [ ]:
n = 10
c = random_cycles2(n)
p = cycle_to_perm(c, n)
def str_p(p):
    return ", ".join(f'{head}->{tail}' for head, tail in enumerate(p))

print(f"c = {c}\np = {str_p(p)}")

In [ ]:
n = 10
points = list(range(n))
while n:
    print(f"{pop(points, 0, n)}, {points[:n-1]}")
    n -= 1

In [ ]:
%%ipytest

def test_Permutation_str():
    n = 10
    for _ in range(100):
        c = random_cycles(n)
        p = cycle_to_perm(c, n)
        assert str(Permutation(p)) == f"cycles : {c}"



In [ ]:

%timeit random_cycles(10_000)
%timeit random_cycles2(10_000)

%timeit random_cycles(100_000)
%timeit random_cycles2(100_000)


# Szorgalmi feladat

Implementáljuk a `RandomNames` osztályt.

A egyedeknek legyen egy `names` property-je, ami olvasáskor a névsort az
eredeti sorrendben adja vissza, és íráskor beállítja az új névsort. A
`RandomNames` legyen iterálható és az `__iter__` metódusa egy végtelen
generátort adjon vissza, ami véletlen sorrendben megy végig a neveken,
ha végig ért akkor újra kezdi újrasorsolt véletlen sorrendben. Az
egyedek legyenek függvényszerűek is (`__call__` metódus). A függvény
hívás eredménye legyen egyetlen találomra választott név a névsorból.

Emlékeztető: Egy osztályban a `@property` dekorátorral tudunk property
mezőt létrehozni. A dekorált függvény szolgál a mező kiolvasára. Ha
írhatóvá szeretnénk tenni a mezőt, akkor a `mezőnév.setter` dekorátort
kell használni.

Pl.
```python
class RandomNames:
    
    @property
    def names(self):
        pass
   
    @names.setter
    def names(self, new_value):
        pass

```

A `names` mező írásakor végezzen ellenőrzést a függvény. Azaz ha nem
sztringekből álló sorozattal hívjuk meg, akkor dobjon `TypeError`-t. A
kapott nevek első betűjét írja át nagy betűvé, a többit viszont írja át
kis betűre. Figyeljünk arra, hogy `rn.names = 'Attila'` hibát kell, hogy
dobjon, de a sorozat nem csak lista lehet, hanem bármi amin végig lehet
iterálni!

Pl.
```python
    rn = RandomNames(['anDoR'])
    rn.names # -> ['Andor']
    rn.names = ['ALADÁR', 'elemér']
    rn.names # -> ['Aladár', 'Elemér']
```

A véletlenszerű sorrend előállításához használhatjuk a `random` modul,
`shuffle` függvényét. Figyeljünk arra, hogy ez elronthatja az eredeti
sorrendet!

Inicializáláskor az egyed kap egy névsort. Ezt kezelje úgy, mintha
`names` tulajdonságot írtuk volna!

Írjunk teszt függvényt! Ez ellenőrizze az inicializálást, ill. `names`
tulajdonság írását, olvasását. Úgy is, hogy a kettő között az iterátort
is használtuk.



In [ ]:
import random


class RandomNames:

    def __init__(self, names):
        self.names = names

    @property
    def names(self):
        return list(self._names)
        pass

    @names.setter
    def names(self, new_value):
        if isinstance(new_value, str):
            raise TypeError
        try:
            new_value = [value.capitalize() for value in  new_value]
        except:
            raise TypeError
        self._names = new_value

    def __iter__(self):
        names = self.names
        while True:
            random.shuffle(names)
            yield from names

    def __call__(self):
        return random.choice(self._names)

    def __str__(self):
        return self.names

    def __repr__(self):
        return f"{type(self).__name__}({self.names})"

In [ ]:
%%ipytest
from collections import Counter

from pytest import raises

def test_typeerror():
    with raises(TypeError):
        RandomNames("Aladár")

    with raises(TypeError):
        RandomNames([1,2,3])

    rn = RandomNames([])
    with raises(TypeError):
        rn.names = [1, 2]

    with raises(TypeError):
        rn.names = "Aladár"

def test_init():
    names = ["AlAdÁr", "jáNos"]
    cap_names = [name.capitalize() for name in names]

    rn = RandomNames(names)
    assert rn.names == cap_names

    names.append("elemér")

    assert rn.names == cap_names

    it = iter(rn)
    next(it)

    assert rn.names == cap_names

    next(it)
    assert rn.names == cap_names


def test_call():
    names = ["AlAdÁr", "jáNos"]
    cap_names = [name.capitalize() for name in names]
    rn = RandomNames(names)
    assert rn() in cap_names
    assert rn.names == cap_names

    n = 1000
    cnt = Counter(rn() for _ in range(2*n))
    assert tuple(cnt.keys()) == tuple(rn.names)
    test_stat = sum(((c-n)**2)/n for c in cnt.values())
    assert test_stat < 1.96**2 ## kb. 5%-os tévedés

def test_names():
    rn = RandomNames([])
    names = ["AlAdÁr", "jáNos"]
    rn.names = names
    assert rn.names == [name.capitalize() for name in names]

def test_randomness():
    """????"""
    pass

In [ ]:
from itertools import islice

rn = RandomNames(("Micimackó", "Füles", "Róbert gida", "tigris", "malacka"))

print("calling `rn`")
for _ in range(10):
    print(f"{rn()=}")

print("iterating over `rn`")
for name in islice(rn, 10):
    print(f"{name=}")

print("iterating over `rn` in pair")
for name1, name2 in islice(zip(rn, rn), 10):
    print(f"{name1=:>20}, {name2=:>20}")


# Numpy (Numerikus python) könyvtár

  - [Numpy bevezető `w3schools`](https://www.w3schools.com/python/numpy/default.asp)


Előadáson volt szó róla. Nagyon sok könyvtár használja (`Pandas`, `Scipy`, `matplotlib`, `statmodels`, stb.).

 Általában, ha ,,tudományos'' számolásról van szó: vektorok, mátrixok, tömbök, akkor a háttérben a `numpy` van.

Legfontosabb típusa az `ndarray` (többdimenziós tömb)

In [ ]:
import numpy as np

In [ ]:
help(np.ndarray)

## `Numpy` tömb létrehozása


Listából, vagy tuple-ból.

Nézzük meg mit kapunk ha lista helyett generátorfüggvényt, `map` vagy `range` objektumot adunk meg


In [ ]:
import numpy as np

In [ ]:
x = np.array([1,2,3], dtype=np.int8)
print(f"{x.shape=}, {x.dtype=}, {x.strides=}, {x.ndim=}")

vagy adott méretű tömböt hozunk létre és azt utólag kitöltjük:

In [ ]:
# x = np.zeros((2,3,4), dtype=np.float64)
x = np.ones((2, 3, 12), dtype=np.int32)

x

In [ ]:
print(f"{x.shape=}, {x.dtype=}, {x.strides=}, {x.ndim=}")

Véletlen értékekkel feltöltött tömb:

In [ ]:
# találomra választott számok a (0,1)-ből
uniform = np.random.uniform(0, 1, size=(10, 10, 10))

# Gauss görbe, normális eloszlás

gauss = np.random.normal(0, 1, size=(10, 10, 10))

# np.random.rand(0, 1, size=(10, 10))

In [ ]:
import matplotlib.pyplot as plt
import math

In [ ]:
sqrt2pi = math.sqrt(2*math.pi)
plt.hist(uniform.reshape(-1), density=True, alpha=0.5, label="uniform")
plt.hist(gauss.reshape(-1)/sqrt2pi, density=True, alpha=0.5, label="Gauss")
plt.legend()
plt.show()

## Műveletek tömbökkel

### Jellemzők kiolvasása:

```
x = np.zeros((10,20))
```

- `x.shape` a tömb méretét adja meg
- `x.ndim` a dimenziók száma (`x.shape` hossza)
- `x.dtype` a tömb elemeinek típusa
- `x.stride`, `x.base`, `x.size` kevésbé érdekes számunkra

### Jellemzők megváltoztatása:

- `x.astype(np.uint8)` új tömböt ad vissza, amiben nem előjeles 8 bites számok fog lesznek.
- `x.reshape(-1)` egy dimenziós tömb az eredmény, a tartalom nem változik.
- `x.T` transzponálás, csak a `strides` paramétert változtatja (gyors).
- `x.transpose(1,0)` transzponálás másképp.

Mit méretű tömböt kapunk a következő sorral? Miért?

```python
x = np.zeros((10,20))
x.reshape((5,2,5,4)).transpose(0,2,1,3).reshape(25, -1)
```

In [ ]:
x = np.zeros((10,20))
x.reshape((5,2,5,4)).transpose(0,2,1,3).reshape(25, -1).shape

In [ ]:
import numpy as np

In [ ]:
x = np.random.normal(0, 1, (4, 6))
print(f"{hex(id(x))=}, {x.shape=}, {x.ndim=}, {x.dtype=}, {x.strides=},  {x.base=}, {x.data=}")
y = x.reshape(-1)
print(f"{hex(id(y))=}, {y.shape=}, {y.ndim=}, {y.dtype=}, {y.strides=},  {hex(id(y.base))=}, {y.data=}")


In [ ]:

y[y<0] = 0
print(x)

In [ ]:
x = np.random.normal(0, 1, (5, 6))
print(f"{hex(id(x))=}, {x.shape=}, {x.ndim=}, {x.dtype=}, {x.strides=},  {x.base=}, {x.data=}")
y = x.astype(np.float32)
print(f"{hex(id(y))=}, {y.shape=}, {y.ndim=}, {y.dtype=}, {y.strides=},  {y.base=}, {y.data=}")

In [ ]:
y[y<0] = 0
print(x)

In [ ]:
x = x.round(3)
print(x)
print(x.transpose(1,0))


## Elemenkénti műveletek

Amit megszoktunk, hogy számokkal működik, tömbökkel (`np.array`) is fog.
pl.


In [ ]:
x = np.random.standard_normal((2, 3))
y = np.random.standard_normal((2, 3))
x, y, x+y, x*y, x-y, x/y, x//y, x**2

Matematikai függvények is alkalmazhatóak, többnyire `np.` előtaggal.

pl.

In [ ]:
x = np.random.standard_normal((2, 3))
np.exp(x), np.abs(x), np.log(np.abs(x))


Ezeknek a függvényeknek van `out` és `where` paraméterük.

In [ ]:
x = np.arange(10).reshape(2,-1)
y = np.ones_like(x)
np.add(y, x, out=y, where=x>5)
print(y)

## Összegzés, szorzás, max, min (redukció)

`for` ciklus helyett egy tömb összegét, maximumát, szorzatát stb. a megfelelő metódus meghívásával is ki lehet számolni.

- Gyorsabb
- Kevesebb hiba lehetőség
- Olvashatóbb

Ezeknek a függvényeknek két szokásos extra paramétere van: `axis`, `keepdims`

Alapértelmezésben a teljes tömböt egy számra redukálják, ha az `axis` meg van adva, akkor az adott tengely mentén redukálnak.

Példák:

In [ ]:
x = np.arange(75).reshape(3,5,5)

print(f"{x.sum()=}")
print(f"{x.sum(axis=1).shape=}")
print(f"{x.sum(axis=(1,2)).shape=}")
print(f"{x.sum(axis=(1,2), keepdims=True).shape=}")


További példák

In [ ]:
x = np.random.uniform(size=(2, 3))

with np.printoptions(precision=4):
    for op in [np.max, np.min, np.sum,  np.cumsum, np.prod, np.cumprod]:
        # op_name = f"np.{op.__name__}"
        print(f"op={op.__name__}")
        print(f"{op(x)=}")
        print(f"{op(x, axis=0)=}")
        if not op.__name__.startswith("cum"):
            print(f"{op(x, axis=0, keepdims=True)=}")
        print("="*50)




Mint mindig ha valamire nem emlékszünk a dokumentáció segít:

pl.
```
help(np.sum)
```

## Véletlenszám generálás

Ezek a függvények az `np.random` modulban vannak

Legfontosabbak:

- `np.random.uniform(a, b, size=(10,10))` `size` méretű tömb, minden elem találomra választott szám `(a,b)`-ből
- `np.random.normal(mu, sigma, size=(10,10))` `size` méretű tömb, minden elem normális eloszlású $\mu$ eltolás $\sigma$ skála paraméterrel.
- `np.random.

In [ ]:
print(f"    {np.random.uniform(0, 1, 5)=}")
print(f"     {np.random.normal(0, 1, 5)=}")
print(f"{np.random.binomial(10, 0.5, 5)=}")
print(f"{np.random.binomial( 1, 0.5, 5)=}")
print(f"{np.random.permutation(5)=}")

Reprodukálhatóság:

In [ ]:
np.random.seed(3)
print(f"{np.random.binomial( 1, 0.5, 5)=}")
np.random.seed(3)
print(f"{np.random.binomial( 1, 0.5, 5)=}")
np.random.seed(3)
print(f"{np.random.binomial( 1, 0.5, 5)=}")


A `seed`-et egyszer szokás beállítani a notebook elején.

## Indexelés

A szokásos `slice` jelölés mellett logikai vektor is lehet index és lista is.

In [ ]:
with np.printoptions(linewidth=110, precision=4):
    x = np.random.normal(0, 1, 10)
    print(f"{x=}")
    print(f"{x>0.2=}")
    print(f"{x[x>0.2]=}")
    print(f"{x[[1,3,9]]=}")


## Broadcasting

Azonos méretű tömböket összeadhatunk, szorozhatunk.

Két tömb `a` és `b` kompatibilis, ha

- `a.ndim=len(a.shape)` és `b.ndim=len(b.shape)` azonos
- és `a.shape[i] == b.shape[i]` vagy az egyik 1


pl. `a = np.zeros((1, 1, 3))` és ` c = np.zeros((2, 3, 3))` kompatibilis, de egyik sem kompatibilis a `c = np.zeros((2, 3, 1))` tömbbel.

Kompatibilis tömbökkel is lehet műveleteket végezni, ahol az alak 1, ott az érték ismétlődik.

In [ ]:
a = np.arange(1,10).reshape(1, 9) ## sorvektor
b = np.arange(1,10).reshape(9, 1) ## oszlopvektor
a*b

Két tömb kompatibilissé tehető, ha néhány 1-est a shape elé írva kompatibilis tömböt kapunk. Ilyen esetben is értelmesek a műveletek.

In [ ]:
a = np.ones(10) ## sorvektor
b = np.ones((10, 1)) ## oszlopvektor
print(f"{a.shape=}, {b.shape=},\n{a*b=}")

a = np.ones(10) ## sorvektor
b = np.ones((10)) ## oszlopvektor
print(f"{a.shape=}, {b.shape=},\n{a*b=}")


Ha új dimenziót akarunk a tömbhöz adni, azt `None`-nal is megtehetjük

In [ ]:
a = np.ones((10,20))
print(f"{a.shape=}, {(a[None]).shape=}")

print(f"{a.shape=}, {(a[:,None]).shape=}")
print(f"{a.shape=}, {(a[...,None]).shape=}")

Olvashatóbb megoldás az `np.expand_dims` függvény használata.

## Tömbök összefűzése

- `np.concatenate`
- `np.stack`

Mindkettőnek hasonló, de a `stack` új dimenziót hoz létre és csak azonos méretű tömböket tud összerakni.


In [ ]:
a = np.ones((1, 5))
b = np.ones((1, 2))

c = np.concatenate((a,b), axis=-1)
print(f"{c.shape=}")

d = np.stack((a, a, a), axis=1)
print(f"{d.shape=}")


# Matplotlib könyvtár

Ez a leggyakrabban használt könyvtár ábrák készítéséhez.

- [Matplotlib bevezető `w3schools`](https://www.w3schools.com/python/matplotlib_intro.asp)

A `pyplot` modult általában `plt` alias-szal importáljuk.

Leggyakrabban használt függvények:

- `plt.plot`
- `plt.scatter`
- `plt.hist`
- `plt.imshow`

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x =  np.random.binomial(1, 0.5, 10)
print(x)
y = np.concatenate(([0], (2*x-1).cumsum()))
plt.plot(y)
plt.show()

Próbáljuk ki mi történik, ha a második argumentum egy string, pl. "o-r", vagy "o:g"

A pontok `x,y` koordinátáját színét és méretét is megadhatjuk:

In [ ]:

x = np.random.randint(100, size=(100))
y = np.random.randint(100, size=(100))
colors = np.random.randint(100, size=(100))
sizes = 10 * np.random.randint(100, size=(100))

plt.grid()
plt.scatter(x, y, c=colors, s=sizes, alpha=0.5, cmap='nipy_spectral')

plt.colorbar()

plt.show()


### Hisztogram

In [ ]:
x = np.random.standard_normal(10000)
plt.hist(x, density=True, bins=np.linspace(-3.5, 3.5, 15))
plt.grid()
plt.show()

A `matplotlib` mellett más ábrakészítő könyvtárak is vannak:

- `seaborn`
- `plotly`
- `plotnine`

# Conway Game of Life


The Game of Life is a cellular automaton created by mathematician John Conway in 1970. The game consists of a board of cells that are either on or off. One creates an initial configuration of these on/off states and observes how it evolves. There are four simple rules to determine the next state of the game board, given the current state:

- **Overpopulation**: if a living cell is surrounded by more than three living cells, it dies.
- **Stasis**: if a living cell is surrounded by two or three living cells, it survives.
- **Underpopulation**: if a living cell is surrounded by fewer than two living cells, it dies.
- **Reproduction**: if a dead cell is surrounded by exactly three cells, it becomes a live cell.


Írjunk egy osztályt a játékhoz, pl. az `__init__` metódus hozza létre a megadott konfigurációnak megfelelő objektumot. Legyen egy `step` metódus, ami a rendszert a következő állapotába viszi és az `__str__` metódus pedig
valahogy ábrázolja az aktuális állapotot.

Tegyük fel, hogy a rács amin a rendszer él, egy $m\times n$-es rács, ahol mindkét irányban ciklikusan körbemegyünk,
azaz a csúcsokat modulo $m$ ill. modulo $n$ tekintjük.


In [ ]:
class ConwayGoL:

    def __init__(self, state):
        self.state = list(state)

    def step(self):
        return self

    def __repr__(self):
        return f"{type(self).__name__}({self.state})"

In [ ]:
import random

init_state = [] ## ???
conway = ConwayGoL(init_state)

conway.step()


A játék állapotának leírásához egy $m\times n$ rács minden pontjáról tudni kell, hogy foglalt-e vagy sem.

```python
m, n = 11, 11
state = [[0]*n for _ in range(m)]
```

Véletlenszerű kezdeti állapot:
```python
state = [[random.randint(0,1) for _ in range(n)] for _ in range(m)]
```

In [ ]:
def random_state(n, m, p):
    return [[int(random.random()<p) for _ in range(m)] for _ in range(n)]

state = random_state(5, 6, 0.2)
print(state)

Szebb megjelenítés?

In [ ]:
def as_matrix(lst, n):
    return [lst[i:i+n] for i in range(0, len(lst), n)]

print('\n'.join(''.join(map(str, line)) for line in state))


In [ ]:
for symbols in [
    "\u2b1c\u2b1b",
    "·♥",
    "🟡🟥"
    ]:
    print('\n'.join(''.join(symbols[x] for x in line) for line in state))


In [ ]:
import matplotlib.pyplot as plt

img = plt.matshow(state, cmap="Pastel1", vmax=1, vmin=0, alpha=0.8)
img.axes.axis("off")
n, m = len(state), len(state[0])
for pos in range(0, n+1):
    img.axes.axhline(y=pos-0.5, color="gray")
for pos in range(0, m+1):
    img.axes.axvline(x=pos-0.5, color="gray")

plt.show()


In [ ]:

def cgol_str(self):
    symbols = "\u2b1c\u2b1b" # ⬜⬛
    return '\n'.join(''.join(symbols[x] for x in line) for line in self.state)

ConwayGoL.__str__ = cgol_str



In [ ]:
conway = ConwayGoL(state)
print(conway)

A `step` metódushoz ki kellene számolni egy adott csúcs foglalt szomszédainak számát `cnt`. Ha ez kész,
akkor az $i$ csúcs új állapota:

$$
    \text{state}_{t+1}[i]=
    \begin{cases}
    1 &\text{Ha $\text{cnt}[i]\in\{2,3\}$ és $\text{state}_t[i]=1$}\\
    1 &\text{Ha $\text{cnt}[i]\in\{3\}$ és $\text{state}_t[i]=0$}\\
    0 &\text{különben}
    \end{cases}
$$

In [ ]:
def newstate(state, count):
    return [int((c==3)|((c==2) & (s==1))) for s, c in  zip(state, count)]

In [ ]:
import ipytest
ipytest.autoconfig()

In [ ]:
%%ipytest

def test_newstate():
    res = [0]*9
    res[2] = 1
    res[3] = 1
    assert newstate([1]*9, list(range(9))) == res
    res = [0]*9
    res[3] = 1
    assert newstate([0]*9, list(range(9))) == res


In [ ]:
def count_neighbors(state):
    delta = [(0,-1), (0, 1), (1,-1), (1,0), (1,1), (-1,-1), (-1,0), (-1,1)]
    m, n = len(state), len(state[0])
    return [ [sum(state[(i+di) % m][(j+dj) % n] for di, dj in delta) for j in range(n)] for i in range(m)]

def cgol_step(self):
    counts = count_neighbors(self.state)
    self.state = [ newstate(line, cnt) for line, cnt in zip(self.state, counts) ]
    return self

ConwayGoL.step = cgol_step

In [ ]:
@classmethod
def cgol_from_random_state(cls, n, m,  p):
    return cls(random_state(n, m, p))

ConwayGoL.from_random_state=cgol_from_random_state

In [ ]:
conway = ConwayGoL.from_random_state(5, 10, 0.2)
print(conway)
print(*count_neighbors(conway.state), sep='\n')

In [ ]:
conway = ConwayGoL.from_random_state(5, 5, 0.25)
print(conway)
print("-"*20)
print(conway.step())

Tudunk-e valami animációszerűt készíteni? Jupyter notebook-ban pl. a következő képpen lehet:

In [ ]:
from ipywidgets import Output
from time import sleep


In [ ]:

out = Output()
display(out)
conway = ConwayGoL.from_random_state(n=21, m=51, p=0.2)

for i in range(50):
    out.clear_output(True)
    with out:
        print(f"After {i} steps:\n{conway}")
    sleep(0.15)
    conway.step()


## Ugyanez `numpy` tömbbel

In [ ]:
import numpy as np

### Random `state`

In [ ]:
def random_state_np(m, n, p):
    return np.random.binomial(1, p, size=(m, n)).astype(np.int8)


In [ ]:
print(random_state_np(11, 21, 0.2))

### `__str__` unicode karakterrel

In [ ]:

symbols_array = np.array(["\u2b1c", "\u2b1b"])

def str_state_np(state):
    return '\n'.join(map(''.join, symbols_array[state]))

In [ ]:
print(str_state_np(random_state(11, 21, 0.2)))

### Szomszédszám `pad`-el

In [47]:
def count_neighbors_np(state, mode='wrap'):
    count = np.pad(state, pad_width=((1,1), (1,1)), mode=mode)
    count = count[2:]+ count[1:-1] + count[:-2]
    count = count[:, 2:] + count[:, 1:-1] + count[:,:-2]
    return count-state

In [ ]:
state = random_state_np(5, 8, 0.2)
print(str_state_np(state))
print(count_neighbors_np(state))

In [49]:
def new_state_np(state, mode='wrap'):
    count = count_neighbors_np(state, mode=mode)
    return ((count == 3)|((count == 2) & (state==1))).astype(np.int8)

In [ ]:
x = np.arange(5)
(x<4)&(x>2)

In [ ]:
state0 = random_state_np(11, 21, 0.2)
state1 = new_state_np(state0)
print(str_state_np(state0))
print('='*50)
print(str_state_np(state1))

In [ ]:
state = random_state_np(11, 21, 0.2)
plt.imshow(state, cmap='Pastel1_r', vmax=1, vmin=0)
plt.xticks(np.arange(state.shape[1]+1)-.5, minor=True)
plt.xticks([])
plt.yticks(np.arange(state.shape[0]+1)-.5, minor=True)
plt.yticks([])
plt.grid(which="minor", color="gray", linestyle='-', linewidth=1)
plt.axis()

for (i, j), cnt in np.ndenumerate(count_neighbors_np(state)):
    plt.text(j, i, str(cnt), ha="center", va="center")


In [ ]:
plt.imshow(state, cmap='Pastel1_r', vmax=1, vmin=0)
plt.xticks(np.arange(state.shape[1]+1)-.5, minor=True)
plt.xticks([])
plt.yticks(np.arange(state.shape[0]+1)-.5, minor=True)
plt.yticks([])
plt.grid(which="minor", color="gray", linestyle='-', linewidth=1)
plt.axis()

for (i, j), cnt in np.ndenumerate(new_state_np(state)):
    plt.text(j, i, str(cnt), ha="center", va="center")


In [ ]:
out = Output()
display(out)
state = random_state_np(n=51, m=21, p=0.2)

for i in range(50):
    out.clear_output(True)
    with out:
        print(f"After {i} steps:\n{str_state_np(state)}")
    sleep(0.15)
    state = new_state_np(state)


## Parancssoros script

Ha parancssorból dolgozunk, akkor valami ilyesmit lehetne tenni

In [ ]:
%%writefile conway.py

import random


def random_state(m, n, p):
    return [ [ int(random.random()<p) for _ in range(n) ] for _ in range(m) ]

def count_neighbors_np(state, mode='wrap'):
    count = np.pad(state, pad_width=((1,1), (1,1)), mode=mode)
    count = count[2:]+ count[1:-1] + count[:-2]
    count = count[:, 2:] + count[:, 1:-1] + count[:,:-2]
    return count-state

def newstate(state, count):
    return [int((c==3)|((c==2) & (s==1))) for s, c in  zip(state, count)]

class ConwayGoL:
    symbols = "\u2b1c\u2b1b"

    def __init__(self, state):
        self.state = list(state)

    def step(self):
        counts = count_neighbors(self.state)
        self.state = [ newstate(line, cnt) for line, cnt in zip(self.state, counts) ]
        return self


    def __str__(self):
        symbols = self.symbols
        return '\n'.join(''.join(symbols[x] for x in line) for line in self.state)

    def __repr__(self):
        return f"{type(self).__name__}({self.state})"

    @classmethod
    def from_random_state(cls, m, n, p):
        return cls(random_state(m, n, p))

    def is_empty(self):
        return not any(any(line) for line in self.state)


def clear_terminal(n):
    print(f"{chr(27)}[{n+1}A", end="")

def main(m=11, n=25, p=0.2, nsteps=10, clear_screen=clear_terminal):
    from time import sleep
    conway = ConwayGoL.from_random_state(m, n, p)
    for i in range(nsteps+1):
        if i>0:
            clear_screen(m)
        print(f"after {i} step:")
        print(conway)
        sleep(0.2)
        conway.step()
        if conway.is_empty():
            break

if __name__ == "__main__":
    main()


Ha valamit már megírtunk és szeretnénk használni, `import`-tal elérhető. Pl.

### Tudunk-e paramétereket adni a python scriptnek?

Amikor egy python scriptet futtatunk, a parancssor (amivel a futást indítottuk) a `sys` modul `argv` változójában érhető el.

In [ ]:
import sys
sys.argv

In [ ]:
! python -c 'import sys; print(sys.argv)' -alma


Egy nagyon egyszerű megoldás, ha minden opciónak a neve a paraméter amit beállít és egyenlőségjel után az értéke:
pl. n=11 m=25 nstep=10 p=0.2

In [ ]:
cmdline = "conway.py -n=11 -m=25 -nstep=10 -p=0.2"
argv = cmdline.split()
params =[param.split("=") for param in argv[1:]]
params

Minden paraméterről tudni kellene, hogy milyen típusú!

In [ ]:
param_types={'-n': int, '-m': int, '-nstep': int, '-p': float}
params = {k.replace("-",""): param_types[k](v)  for k, v in (param.split("=") for param in argv[1:])}
params

Ezek után a `main` függvényt a megadott paraméterekkel meg tudjuk hívni:

```
    main(**params)
```
Mi van a `default` értékekkel, `help`-pel stb.

Ezeket mind meg tudnánk írni, de nem kell. Van kész megoldás `python`-ban.

Az `argparse` könvytár mindent megcsinál, ami nekünk kell.

In [ ]:
import argparse

help(argparse)

A `conway.py` file végét cseréljük le erre.
```python
if __name__ == "__main__":
    import argparse
    
    parser = argparse.ArgumentParser(description='Conways Game of Life')

    parser.add_argument(
        '-n', '--nrows',
        type=int,
        default=11,
        help='number of rows'
        )

    parser.add_argument(
        '-m', '--ncols',
        type=int,
        default=25,
        help='number of columns'
        )

    parser.add_argument(
        '-p', '--density',
        type=float,
        default=0.2,
        help='initial density')
    
    parser.add_argument(
        '--nsteps',
        type=int,
        default=10,
        help='steps to display'
        )

    args = parser.parse_args()
    print(args)
    
    main(n=args.ncols, m=args.nrows, p=args.density, nsteps=args.nsteps)
```
    

In [ ]:
# import importlib
# importlib.reload(conway)

In [ ]:
import conway

out1 = Output()
display(out1)

with out1:
    conway.main(clear_screen=lambda n: out1.clear_output(True))

Az `argparse` könyvtár nem a legkényelmesebb. Alternatívák:

- [Docopt](http://docopt.org/)
- [Click](https://pypi.org/project/click/)
- [clize](https://github.com/epsy/clize)

és még sok másik is!

## Itt is használhattunk volna dekorátort


A `ConwayGoL` példában utólag adtunk metódusokat az osztályunkhoz. Ezt is megtehettük volna dekorátorral.  

In [ ]:
def conway_method(f):
    setattr(ConwayGoL, f.__name__, f)
    return f


@conway_method
def dummy_method(self):
    print("this is a message from the new method!")

c = ConwayGoL([])
c.dummy_method()

Azt is megtehettük volna, hogy a osztály nincs beleégetve a kódba.

In [ ]:
def new_method(cls):
    def decorator(f):
        setattr(cls, f.__name__, f)
        return f
    return decorator

@new_method(ConwayGoL)
def dummy_method(self):
    print("Note that the old value of dummy_method is overwritten!")

In [ ]:
c.dummy_method()

## Néhány minta

In [82]:
import urllib.request as request
import zipfile

url = "https://conwaylife.com/patterns/all.zip"
url = "https://conwaylife.com/patterns/83p7h1v1.cells"
with request.urlopen(url) as file:
    btext = file.read()
    # with open("/tmp/all.zip", "wb") as file:
    #     file.write(response.read())


In [85]:
text = btext.decode("utf-8")
print(text)
data = [[0 if c == "." else 1 for c in line] for line in text.splitlines() if not line.startswith("!")]
# print(np.array(data))

! Lobster (spaceship)
! Matthias Merzenich
! https://conwaylife.com/wiki/Lobster_(spaceship)
! https://conwaylife.com/patterns/83p7h1v1.cells
...........OOO............
.............O............
........OO..O.............
........OO................
............OO............
...........OO.............
..........O..O............
..........................
........O..O..............
.......O...O..............
......O.OOO...............
.....O....................
.....O.............O.O..OO
......O.............OO.O.O
.OO.............OO..O....O
O..OO..OO......O...O......
.....O..O......O......OO..
.........OO....O.O....OO..
..O...O...O.....O.........
......OO....O..O..........
.O.O.....O...OO...........
OO........O...............
.....O....O...............
.......O...O..............
....OO.....O..............
....O.....O...............


In [30]:
! ls /tmp/all.zip

/tmp/all.zip


In [93]:
# ! unzip -l /tmp/all.zip | grep gun

In [94]:
# ! unzip /tmp/all.zip bigun.cells

In [95]:
# def decode_line(text):
#     return [0 if c == '.' else 1 for c in text.strip()]

# with open("bigun.cells", "r") as file:
#     lines = file.readlines()

# print("".join(line for line in lines if line.startswith("!")))
# data = [decode_line(line.strip()) for line in lines if not line.startswith("!")]
  
# row_len = max((len(line) for line in data), default=0)
# print(row_len)
# data = [line + [0]*(row_len-len(line)) for line in data]

In [51]:
from ipywidgets import Output
from time import sleep
import numpy as np


In [53]:
symbols_array = np.array(["\u2b1c", "\u2b1b"])

def str_state_np(state):
    return '\n'.join(map(''.join, symbols_array[state]))

def count_neighbors_np(state, mode='wrap'):
    count = np.pad(state, pad_width=((1,1), (1,1)), mode=mode)
    count = count[2:]+ count[1:-1] + count[:-2]
    count = count[:, 2:] + count[:, 1:-1] + count[:,:-2]
    return count-state

def new_state_np(state, mode='wrap'):
    count = count_neighbors_np(state, mode=mode)
    return ((count == 3)|((count == 2) & (state==1))).astype(np.int8)

In [91]:

out = Output()
display(out)

state = np.pad(np.array(data), ((20, 0), (0, 20)), mode='constant')

for i in range(100):
    out.clear_output(True)
    with out:
        print(f"After {i} steps:\n{str_state_np(state)}")
    sleep(0.15)
    state = new_state_np(state, "constant")
    if (state == 0).all():
        break


Output()